In [2]:
%load_ext autoreload
%autoreload

# Load the required modules
import sys
sys.path.append("../..")

import os

import tqdm
import numpy as np

from datasetGenerator import DCASE2018
from Binarizer import Binarizer
from Encoder import Encoder
import pickle

from evaluation_measures import event_based_evaluation
from dcase_util.containers import MetaDataContainer

Using TensorFlow backend.


In [3]:
# Load the pkl file
# pkl_file='../test_predictions/multitask_100epochs.pkl'
# pkl_file='../test_predictions/dico_10epochs_MIL_cnn.pkl'
pkl_file="results/testing/eval_880_dico_10epochs_MIL_cnn_classif_cnn_100epochs_f1_0.8584_valf1_0.8286.pkl"
#pkl_file="results/testing/thomas_last.pkl"
#pkl_file='../test_predictions/dico_10epochs_MIL_cnn_classif_cnn_100epochs_f1_0.8584_valf1_0.8286.pkl' # best: 24.46
# pkl_file='../test_predictions/dico_10epochs_MIL_cnn_classif_cnn_70epochs_final_xtrain_f1_0.8762.pkl'
# pkl_file='../test_predictions/dico_10epochs_MIL_cnn_crnn_adaptation.pkl'
# pkl_file='../test_predictions/test_thresh_0_5_dico_10epochs_MIL_cnn_classif_cnn_100epochs_f1_0.8584_valf1_0.8286.pkl'

# pkl_file='../eval_predictions/eval_880_dico_10epochs_MIL_cnn_classif_cnn_100epochs_f1_0.8584_valf1_0.8286.pkl'

with open(pkl_file, 'rb') as f:
    data = pickle.load(f)

In [6]:
# print(data["Y0UN02RkbG_U_7.000_17.000"])

In [4]:
# format the curves to be used by the Encoder (need ALL curves)
featRoot = "/baie/corpus/DCASE2018/task4/FEATURES/"
metaRoot = "/baie/corpus/DCASE2018/task4/metadata"
feat =  ["mel"]
normalizer = None

dataset = DCASE2018(
        featureRoot=featRoot,
        metaRoot=metaRoot,
        features=feat,
        expandWithUod=False,
        validationPercent=0.2,
        normalizer=normalizer
    )


mel


In [5]:
testListFile = [f[:-8] for f in dataset.testFileList]
nbFrame = 431

# rearange list of results
results = []
lfn = []
for f in testListFile:
    info = data[f]
    
    curves = np.array([[0]*dataset.nbClass for _ in range(nbFrame)], dtype=np.float32)
    
    for cls in info:
        index = DCASE2018.class_correspondance[cls]
        curves[:,index] = info[cls]
        
    results.append(curves)
    lfn.append(f)

results = np.array(results)

print(testListFile[:5])
print(lfn[:5])

KeyError: 'Y0UN02RkbG_U_7.000_17.000'

In [10]:
encoder = Encoder()
#segments = encoder.encode(results, method="hysteresis", high=0.1, low=0.05)#, smooth="smoothMovingAvg")   # 6.66 %
#segments = encoder.encode(results, method="hysteresis", high=0.05, low=0.05)#, smooth="smoothMovingAvg")  # 20.75 %
segments = encoder.encode(results, method="hysteresis", high=0.07, low=0.07, smooth="smoothMovingAvg", window_len=19)  # 21.50 %

#segments = encoder.encode(results, method="hysteresis", high=0.07, low=0.07)#, smooth="smoothMovingAvg")
toEvaluate = encoder.parse(segments, dataset.testFileList)

print("perform evaluation ...")                                                                                                                                                                                                                                   
with open("toEvaluate.csv", "w") as f:                                                                                                                                                                                                                            
    f.write("filename\tonset\toffset\tevent_label\n")                                                                                                                                                                                                             
    f.write(toEvaluate)                                                                                                                                                                                                                                           

perso_event_list = MetaDataContainer()                                                                                                                                                                                                                            
perso_event_list.load(filename="toEvaluate.csv")                                                                                                                                                                                                                  

ref_event_list = MetaDataContainer()                                                                                                                                                                                                                              
ref_event_list.load(filename=dataset.meta_test)                                                                                                                                                                                                                   

event_based_metric = event_based_evaluation(ref_event_list, perso_event_list)                                                                                                                                                                                     


Smooting using the smooth moving average algorithm
perform evaluation ...


NameError: name 'score' is not defined

In [8]:
# Load the evaluation set
featEvalPath = "/baie/corpus/DCASE2018/task4/FEATURES/eval/mel"
featEvalList = os.listdir(featEvalPath)

# load the meta data ----
metaPath = "/baie/corpus/DCASE2018/task4/metadata/eval.csv"
with open(metaPath, "r") as metaFile:
    metadata = metaFile.read().splitlines()[1:]
    
metadata = [i.split("\t") for i in metadata]

# load the features
featEval = []
for file in tqdm.tqdm(featEvalList):
    path = os.path.join(featEvalPath, file)
    feature = np.load(path)
       
    # preprocessing
    feature = np.expand_dims(feature, axis=-1)
    featEval.append(feature)
    
featEval = np.array(featEval)
featEval.shape

100%|██████████| 880/880 [00:00<00:00, 1298.13it/s]


(880, 64, 431, 1)

In [9]:
tmpListFile = [f[:-8] for f in featEvalList]
print(tmpListFile[:5])
print(list(data.keys())[:5])
nbFrame = 431

# rearange list of results
results = []
for f in tmpListFile:
    info = data[f]
    
    curves = np.array([[0]*dataset.nbClass for _ in range(nbFrame)], dtype=np.float32)
    
    for cls in info:
        index = DCASE2018.class_correspondance[cls]
        curves[:,index] = info[cls]
        
    results.append(curves)

results = np.array(results)
results.shape

['YX-jJxA53SFI_130.000_140.000', 'YBJimHxqGfig_50.000_60.000', 'YC7TihoHf_hM_20.000_30.000', 'Y-wQ8El8J_dE_30.000_40.000', 'Y63Qy15I3QGE_30.000_40.000']
['Y6UY-HKyolKk_30.000_40.000', 'YKNFHL6rv_M4_50.000_60.000', 'YEmPv3XBRi-w_50.000_60.000', 'YKkPrgFi9X6U_20.000_30.000', 'Yv-J-jzNonoY_40.000_50.000']


(880, 431, 10)

In [11]:
encoder = Encoder()
# segments = encoder.encode(results, method="hysteresis", high=0.1, low=0.05, smooth="smoothMovingAvg")   # 6.66 %
#segments = encoder.encode(results, method="hysteresis", high=0.05, low=0.05)#, smooth="smoothMovingAvg")  # 20.75 %
segments = encoder.encode(results, method="hysteresis", high=0.07, low=0.07, smooth="smoothMovingAvg", window_len=19)  # 21.50 %
#segments = encoder.encode(results, method="hysteresis", high=0.07, low=0.07)#, smooth="smoothMovingAvg")
toEvaluate = encoder.parse(segments, tmpListFile)

print("perform evaluation ...")                                                                                                                                                                                                                           
with open("toto_toEvaluate.csv", "w") as f:                    
    f.write("filename\tonset\toffset\tevent_label\n")                                                                                                                                                                                                             
    f.write(toEvaluate)                                                                                                                                                                                                                                           


Smooting using the smooth moving average algorithm
perform evaluation ...


In [ ]:
submission_fname='../submissions/0/sub0_10epochs_MIL_cnn_classif_cnn_100epochs_f1_0.8584_valf1_0.8286_macrof1_0.2446.txt'
print("Saving final results in %s"%submission_fname)                                                                                                                                                                                                                
with open(submission_fname, "w") as f:                                                                                                                                                                                                                             
    f.write(str(event_based_metric))

In [ ]:
for cls in segments[0]:
    print("cls: ", cls, segments[0][cls])

In [ ]:
print(toEvaluate)